Codes in this jupyter notebook shows work on the entire ZVA05 table (SAP data).

The overall goal is to see if segmenting dealers by purchase behavior (patient vs impatient--# of days between order creation date and requested delivery date) and reorder cadence (# of days between two orders). 

Dealers: from sales office 'HYB' and 'STD,' excluding liquidation sales from three dealers.

Main conclusions:
- need to set up a time cutoff for ZVA05 data.
- there is no significant correlation between purchase behavior and order quantity; nor between reorder cadence and order quantity. 

__#Notes after modelling: correlation coefficient doesn't work on this dataset since it's heteroskedastic.__

### re-import data from ZVA05 + PRODUCT + CUSTOMER
<br> - remove liquidation sale from customers ACK, GSA and TJX
<br> - Active SL
<br> - Order type: not returns
<br> - Not rejected
<br> - SalesOffice == HYB + STD

In [2]:
import pandas as pd
import numpy as np
import copy

In [13]:
import pyodbc
conn = pyodbc.connect("Driver={ODBC Driver 13 for SQL Server};Server=yetidb01.database.windows.net;database=YETISQLDW01;uid=ezeng;PWD=Ed1tH2EnG#")
cursor = conn.cursor()

query = ("""
WITH ZVA05 AS
(
	(
		SELECT * FROM [EDW].[sap_zva05_fct] 
		WHERE SL_Status = 'Active SL' 
		AND Reason_for_Rejection IS NULL
		AND SalesOffice = 'STD' 
 		AND Order_Type NOT IN ('ZARF', 'ZARM', 'ZEG2', 'ZERF', 'ZERM', 'ZRE')
        AND Sold_to_Party NOT IN ('6610', '91860', '0000107894', '0000114883', '0000108654')
        AND Req_Delivery_Date >= '2017-01-01' AND Req_Delivery_Date <= '2018-05-31'
        AND Document_Date >= '2017-01-01' AND Document_Date <= '2018-05-31'
	) UNION (
		SELECT * FROM [EDW].[sap_zva05_fct] 
		WHERE SL_Status = 'Active SL'
		AND Reason_for_Rejection IS NULL
		AND SalesOffice = 'HYB'
		AND Order_Type NOT IN ('ZARF', 'ZARM', 'ZEG2', 'ZERF', 'ZERM', 'ZRE')
        AND Sold_to_Party NOT IN ('6610', '91860', '0000107894', '0000114883', '0000108654')
        AND Req_Delivery_Date >= '2017-01-01' AND Req_Delivery_Date <= '2018-05-31'
        AND Document_Date >= '2017-01-01' AND Document_Date <= '2018-05-31'
	)
),
PRODUCT AS (
	SELECT ProductID, ProductIDOLD, ProductCategory, MasterSKU, Color, CurrentFlag
	FROM [EDW].[PRODUCT_DIM]
	WHERE ISCore = 'TRUE'
),
CUSTOMER AS (
	SELECT CustomerID, Customer, City, State, Zipcode, Country, District
	FROM [EDW].[CUSTOMER_DIM]
)
SELECT ZVA05.*, PRODUCT.*, CUSTOMER.*
FROM ZVA05
INNER JOIN CUSTOMER ON ZVA05.Sold_to_Party = CUSTOMER.CustomerID
INNER JOIN PRODUCT ON ZVA05.Material = PRODUCT.ProductID OR ZVA05.Material = PRODUCT.ProductIDOLD
ORDER BY Document_Date ASC;
"""
)

In [14]:
new_df = pd.read_sql(query, conn)

In [6]:
new_df.shape

(635947, 69)

In [7]:
new_df.head(2)

,PONumber,Sales_document,SD_Item,Sold_to_Party,Sold_to_Party_Name,ShipToParty,ShipToPartyAddress,BillToParty,BillToPartyAddress,SalesGroup,...,MasterSKU,Color,CurrentFlag,CustomerID,Customer,City,State,Zipcode,Country,District
0,110757-053,0010000013,000010,0000101539,Cavenders,0000204337,0000050385,0000101539,0000080190,STD,...,Flip 12,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO
1,110757-063,0010000011,000010,0000101539,Cavenders,0000204317,0000050341,0000101539,0000080190,STD,...,Flip 12,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO


In [30]:
new_df.columns

Index(['PONumber', 'Sales_document', 'SD_Item', 'Sold_to_Party',
       'Sold_to_Party_Name', 'ShipToParty', 'ShipToPartyAddress',
       'BillToParty', 'BillToPartyAddress', 'SalesGroup', 'SalesOffice',
       'Sales_Org', 'Order_Type', 'Product_Hierarchy', 'Material',
       'Material_Description', 'Unit_Price', 'Order_Qty',
       'Net_Value_item_level', 'Confirmd_Qty_Item', 'Confirmed_Qty_Schl_Line',
       'Document_Date', 'Req_Delivery_Date', 'Goods_Issue_Date',
       'Delivery_Date', 'Original_Promise_Date', 'Original_Promised_Qty',
       'Overall_Credit_Status', 'Delivery_Block', 'Billing_Block',
       'Schl_Line_Number', 'Reason_for_Rejection', 'ReasonForRejectionDesc',
       'Rejected_Date', 'Overall_Delivery_status_Header',
       'Overall_Delivery_status_Item', 'Overall_Order_Status_Header',
       'Overall_Order_Status_Item', 'OrdHedCreatedDate', 'OrdHedCreatedBy',
       'OrdLineCreatedDate', 'OrdLineCreatedBy', 'Order_status', 'Block_flag',
       'Line_Req_Deliv_Dat

In [17]:
type(new_df['Document_Date'][2])

datetime.date

### calculate the intervals between order creation date and requested delivery date

In [18]:
new_df['waiting_days'] = new_df.apply(lambda _: '', axis=1)
print("Adding new column 'waiting_days'")
new_df.head(5)

Adding new column 'waiting_days'


,PONumber,Sales_document,SD_Item,Sold_to_Party,Sold_to_Party_Name,ShipToParty,ShipToPartyAddress,BillToParty,BillToPartyAddress,SalesGroup,...,Color,CurrentFlag,CustomerID,Customer,City,State,Zipcode,Country,District,waiting_days
0,111039-042,0010000000,000010,0000101539,Cavenders,0000204385,0000050487,0000101539,0000080190,STD,...,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO,
1,110757-053,0010000013,000010,0000101539,Cavenders,0000204337,0000050385,0000101539,0000080190,STD,...,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO,
2,110757-061,0010000010,000010,0000101539,Cavenders,0000204379,0000050477,0000101539,0000080190,STD,...,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO,
3,110757-063,0010000011,000010,0000101539,Cavenders,0000204317,0000050341,0000101539,0000080190,STD,...,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO,
4,110757-067,0010000012,000010,0000101539,Cavenders,0000204384,0000050485,0000101539,0000080190,STD,...,Field Tan,Y,0000101539,Cavenders,Tyler,TX,75703-5241,US,WACO,


In [19]:
import sys
import time

# calculate a column of waiting periods at the end of each order
def update_waiting_days():
    df = new_df.copy()
    for row_index, row in df.iterrows():
        interval = df.loc[row_index,'Line_Req_Deliv_Date'] - df.loc[row_index,'Document_Date']
        df.loc[row_index,'waiting_days'] = interval.days
        if row_index%10000 == 0:
            sys.stdout.write('\r'+"{0:.3%}".format(row_index/len(new_df)))
            sys.stdout.flush()
    return df

df = update_waiting_days()

99.393%

In [21]:
df.shape

(603664, 70)

In [22]:
df['waiting_days'].describe()

count    603664.000000
mean         17.827959
std          31.254309
min        -360.000000
25%           0.000000
50%           0.000000
75%          28.000000
max         387.000000
Name: waiting_days, dtype: float64

In [23]:
df.columns

Index(['PONumber', 'Sales_document', 'SD_Item', 'Sold_to_Party',
       'Sold_to_Party_Name', 'ShipToParty', 'ShipToPartyAddress',
       'BillToParty', 'BillToPartyAddress', 'SalesGroup', 'SalesOffice',
       'Sales_Org', 'Order_Type', 'Product_Hierarchy', 'Material',
       'Material_Description', 'Unit_Price', 'Order_Qty',
       'Net_Value_item_level', 'Confirmd_Qty_Item', 'Confirmed_Qty_Schl_Line',
       'Document_Date', 'Req_Delivery_Date', 'Goods_Issue_Date',
       'Delivery_Date', 'Original_Promise_Date', 'Original_Promised_Qty',
       'Overall_Credit_Status', 'Delivery_Block', 'Billing_Block',
       'Schl_Line_Number', 'Reason_for_Rejection', 'ReasonForRejectionDesc',
       'Rejected_Date', 'Overall_Delivery_status_Header',
       'Overall_Delivery_status_Item', 'Overall_Order_Status_Header',
       'Overall_Order_Status_Item', 'OrdHedCreatedDate', 'OrdHedCreatedBy',
       'OrdLineCreatedDate', 'OrdLineCreatedBy', 'Order_status', 'Block_flag',
       'Line_Req_Deliv_Dat

In [6]:
# order creation predating requested delivery date -- negative intervals
# save negative interval data points for future reference

def record_negative_wait_periods():
    neg_intervals = df[df['waiting_days'] < 0]
    neg_interval_orders = []
    for i, row in neg_intervals.iterrows():
        if row[69] < 0:
            neg_interval_orders.append(row[1])  # append Sales_document to list
    print(len(neg_interval_orders))
    return neg_interval_orders
neg_interval_orders = record_negative_wait_periods()
%store neg_interval_orders

30390
Stored 'neg_interval_orders' (list)


In [24]:
# remove invalid data points (negative intervals)
df.loc[df['waiting_days']>=0]['waiting_days'].describe()

count    573274.000000
mean         19.118156
std          31.129744
min           0.000000
25%           0.000000
50%           0.000000
75%          30.000000
max         387.000000
Name: waiting_days, dtype: float64

In [7]:
print("total number of records:", len(df))
print("\rnumber of records with 0-day waiting interval:", len(df.loc[df['waiting_days']==0]), "\t{0:.3%}".format(len(df.loc[df['waiting_days']==0])/len(df)) )
print("\rnumber of records with at least 1 day waiting interval:", len(df[df['waiting_days'] > 0]), "\t{0:.3%}".format(len(df.loc[df['waiting_days']>0])/len(df)) )

total number of records: 602653
number of records with 0-day waiting interval: 309312 	51.325%
number of records with at least 1 day waiting interval: 262951 	43.632%


### Check Order_Qty's Pearson correlation with other numerical features
correlation only measures linear relationships. This step gives a rough estimate for the potential of SLR (simple linear regression) and identify duplicated features that should be removed: 
- Net_Value_item_level, Net_Value, Net_Value_Actual
- Confirmd_Qty_Item, Confirmed_Qty_Schl_Line
- Confirmed_Line_Quantity

In [672]:
df.corr()['Order_Qty']

Unit_Price                -0.125719
Order_Qty                  1.000000
Net_Value_item_level       0.769676
Confirmd_Qty_Item          0.997780
Confirmed_Qty_Schl_Line    0.997642
Original_Promised_Qty      0.383754
Confirmed_Line_Quantity    0.997664
Confirmed_Line_Value       0.771281
Net_Value                  0.771281
Net_Value_Actual           0.769676
waiting_days               0.018247
Name: Order_Qty, dtype: float64

###  Segment orders by buying behaviors (patient vs impatient)
This segment is based on the number of days between Order Creation Date (Document_date) and Requested Delivery Date (Line_Req_Delivery_Date), reflected in the value of 'waiting_days' column.
- impatient: 0 day between Order Creation Date (Document_date) and Requested Delivery Date (Line_Req_Delivery_Date)
- patient: 1 or more days 

In [26]:
def append_wait_indicator():
    df['wait'] = ''
    df.loc[df['waiting_days'] > 0, 'wait'] = 1
    df.loc[df['waiting_days'] == 0,'wait'] = 0
    print("Appended binary indicator to 'wait' column")

In [27]:
append_wait_indicator()

Appended binary indicator to 'wait' column


In [28]:
df.sample(10)

,PONumber,Sales_document,SD_Item,Sold_to_Party,Sold_to_Party_Name,ShipToParty,ShipToPartyAddress,BillToParty,BillToPartyAddress,SalesGroup,...,CurrentFlag,CustomerID,Customer,City,State,Zipcode,Country,District,waiting_days,wait
591191,TK ORDER,0011188507,000150,0000101174,Ketchie Outdoors,0000101174,0000077384,0000421163,0000060910,STD,...,Y,0000101174,Ketchie Outdoors,Granite Quarry,NC,28072,US,NCE,0,0
102073,BRSS175YTI - #2,0020003145,000160,0000103522,Mast General Store,0000103522,0000084386,0000103522,0000084386,STD,...,Y,0000103522,Mast General Store,Valley Crucis,NC,28691,US,NCE,17,1
419601,2972,0010705727,000080,0000100898,Dixie Pickers,0000100898,0000096082,0000100898,0000096082,STD,...,Y,0000100898,Dixie Pickers,Memphis,TN,38119,US,MEMPH,0,0
419279,8225751,0010703732,000060,0000100710,Scheels,0000202509,0000056977,0000100710,0000092038,HYB,...,Y,0000100710,Scheels,Fargo,ND,58103,US,ILWI,0,0
312875,1571835047,0010436835,000230,0000101886,Rural King Supply,0000205276,0000052743,0000410011,0000060920,STD,...,Y,0000101886,Rural King Supply,Mattoon,IL,61938-6643,US,ILWI,1,1
285144,ASAP 080717,0010401676,000010,0000105916,Clark Custom Baling & Farming,0000105916,0000089380,0000105916,0000089380,STD,...,Y,0000105916,Clark Custom Baling & Farming,Bayard,IA,50029-8502,US,MINN,0,0
369912,BRIAN101717,0010535993,000040,0000102466,Hopps Sound & Electric,0000102466,0000082154,0000102466,0000082154,STD,...,Y,0000102466,Hopps Sound & Electric,Mcpherson,KS,67460-4321,US,KSMO,0,0
269150,FPB 10380,0010373654,000170,0000102974,Island WaterSports,0000102974,0000083234,0000102974,0000083234,STD,...,Y,0000102974,Island WaterSports,Deerfield Beach,FL,33441-3702,US,FLS,89,1
129434,JUN17,0010123017,000550,0000100981,Backwater Trading Company,0000100981,0000097820,0000420970,0000060832,STD,...,Y,0000100981,Backwater Trading Company,Edgartown,MA,02539,US,BOSTON,39,1
419074,723401W3622,0010707927,000010,0000101399,DO It Best Corp,0000204017,0000050199,0000410013,0000061036,HYB,...,Y,0000101399,DO It Best Corp,Fort Wayne,IN,46801-0868,US,INKY,8,1


In [31]:
data = df.copy()

#### Check patient vs impatient segmentation's (linear) correlation with Order_Quantity

In [32]:
def check_wait_corr():
    mini_df = data[['Order_Qty','wait']]
    mini_df['wait'] = pd.to_numeric(mini_df['wait'])
    return mini_df
check_wait_corr().corr()

C:\Users\Edith.Zeng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Order_Qty,wait
Order_Qty,1.000000,0.049286
wait,0.049286,1.000000


<br> Check patient vs impatient segmentation's within-in group mean & variance

In [33]:
check_wait_corr().groupby('wait').mean()

C:\Users\Edith.Zeng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Order_Qty
wait,
0.0,13.700849
1.0,22.078648


In [34]:
check_wait_corr().groupby('wait').var()

C:\Users\Edith.Zeng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Order_Qty
wait,
0.0,5015.560101
1.0,9675.070777


In [546]:
mini_df = data[['Order_Qty','wait','month']]
mini_df['wait'] = pd.to_numeric(mini_df['wait'])

C:\Users\Edith.Zeng\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [547]:
mini_df.groupby(['wait', 'month']).var()

Order_Qty
wait month              
0.0  1       3197.923650
     2      16287.362419
     3       6233.470966
     4       5531.990268
     5       2585.579057
     6       1998.035262
     7       5012.418881
     8       3694.364251
     9       3926.808642
     10      5594.301410
     11      4845.511998
     12      4782.771129
1.0  1      27210.888729
     2      10189.447433
     3       7986.667245
     4       3676.314904
     5       4156.196857
     6      13792.404083
     7       4572.125535
     8      10622.577763
     9      18360.235344
     10     39357.430319
     11     27499.275733
     12     82834.848391

#### Incorporate order creation month to increase the number of grouping from 2 (patient/impatient) to 24 (patient/impatient + 12 month)

In [549]:
t=mini_df.groupby(['wait', 'month']).mean()

In [553]:
t.reset_index().corr()

,wait,month,Order_Qty
wait,1.000000,0.000000,0.505678
month,0.000000,1.000000,0.352288
Order_Qty,0.505678,0.352288,1.000000


In [554]:
t

Order_Qty
wait month            
0.0  1       14.621430
     2       17.177771
     3       15.766922
     4       15.100300
     5       11.850371
     6       10.921122
     7       11.954580
     8       12.202798
     9       12.080731
     10      14.538863
     11      16.582096
     12      13.773953
1.0  1       57.053330
     2       21.065201
     3       19.903407
     4       15.017628
     5       16.846264
     6       28.115474
     7       13.881933
     8       24.780300
     9       40.657912
     10      61.252830
     11      57.928828
     12     132.214636

### Check Order_Qty's Pearson correlations with other categorical features

In [2]:
data.shape

(602653, 71)

In [3]:
data = data[['Sales_document', 'Sold_to_Party', 'SalesOffice', 'Order_Type',
       'Unit_Price', 'Order_Qty', 'Document_Date', 'Req_Delivery_Date',
       'Goods_Issue_Date', 'Delivery_Date', 'Overall_Credit_Status',
       'Delivery_Block', 'Billing_Block', 'Block_flag', 'Line_Req_Deliv_Date',
       'Confirmed_Line_Quantity', 'Confirmed_Line_Value',
       'Overall_Credit_Status_Desc', 'Net_Value_Actual', 'ProductCategory',
       'MasterSKU', 'Color', 'CurrentFlag', 'CustomerID', 'Customer', 'City',
       'State', 'Zipcode', 'Country', 'District', 'waiting_days', 'wait']]

In [ ]:
# identify nulls
data.isnull().sum()

In [5]:
data.shape

(602653, 32)

In [6]:
# clean up nulls
data.iloc[:,10:14] = data.iloc[:,10:14].fillna("0")  # Overall_Credit_Status, Delivery_Block, Billing_Block, Block_flag
data.iloc[:,26] = data.iloc[:,26].fillna("0")   # state
data.dropna(how='any',axis='rows',inplace=True) # district

In [7]:
data.isnull().sum()

Sales_document                0
Sold_to_Party                 0
SalesOffice                   0
Order_Type                    0
Unit_Price                    0
Order_Qty                     0
Document_Date                 0
Req_Delivery_Date             0
Goods_Issue_Date              0
Delivery_Date                 0
Overall_Credit_Status         0
Delivery_Block                0
Billing_Block                 0
Block_flag                    0
Line_Req_Deliv_Date           0
Confirmed_Line_Quantity       0
Confirmed_Line_Value          0
Overall_Credit_Status_Desc    0
Net_Value_Actual              0
ProductCategory               0
MasterSKU                     0
Color                         0
CurrentFlag                   0
CustomerID                    0
Customer                      0
City                          0
State                         0
Zipcode                       0
Country                       0
District                      0
waiting_days                  0
wait    

#### Extract year and month of each row's order date

In [8]:
import datetime
def add_year_month_cols():
    data['year'] = data.apply(lambda _: '', axis=1)
    data['month'] = data.apply(lambda _: '', axis=1)
    print(data.columns)
def extract_order_year_month():
    data['Document_Date'] = pd.to_datetime(data['Document_Date'])
    data['year'], data['month'] = data['Document_Date'].dt.year, data['Document_Date'].dt.month

In [9]:
add_year_month_cols()
extract_order_year_month()
data.sample(5)

Index(['Sales_document', 'Sold_to_Party', 'SalesOffice', 'Order_Type',
       'Unit_Price', 'Order_Qty', 'Document_Date', 'Req_Delivery_Date',
       'Goods_Issue_Date', 'Delivery_Date', 'Overall_Credit_Status',
       'Delivery_Block', 'Billing_Block', 'Block_flag', 'Line_Req_Deliv_Date',
       'Confirmed_Line_Quantity', 'Confirmed_Line_Value',
       'Overall_Credit_Status_Desc', 'Net_Value_Actual', 'ProductCategory',
       'MasterSKU', 'Color', 'CurrentFlag', 'CustomerID', 'Customer', 'City',
       'State', 'Zipcode', 'Country', 'District', 'waiting_days', 'wait',
       'year', 'month'],
      dtype='object')


,Sales_document,Sold_to_Party,SalesOffice,Order_Type,Unit_Price,Order_Qty,Document_Date,Req_Delivery_Date,Goods_Issue_Date,Delivery_Date,...,Customer,City,State,Zipcode,Country,District,waiting_days,wait,year,month
582800,0011172952,0000101318,STD,OR,23.99,8.0,2018-05-11,2018-05-11,2018-05-16,2018-05-18,...,Mountain High Outfitters,Birmingham,AL,35222-1321,US,ALA,0,0,2018,5
352208,0010506812,0000102261,STD,OR,11.99,6.0,2017-09-28,2017-09-28,2017-11-03,2017-11-08,...,Sunshine ACE Hardware,Bonita Springs,FL,34135-4232,US,FLS,0,0,2017,9
98130,0020002324,0000101035,STD,QT,23.99,4.0,2017-04-20,2017-06-01,2017-06-08,2017-06-12,...,Binks Outfitters,Franklin,TN,37065-0289,US,NASH,53,1,2017,4
524576,0011011747,0000101164,STD,OR,17.99,6.0,2018-03-12,2018-03-12,2018-03-15,2018-03-15,...,Friedmans Home Improvement,Santa Rosa,CA,95407-8222,US,SFCA,0,0,2018,3
144844,0010156216,0000103396,STD,OR,194.99,1.0,2017-05-17,2017-05-17,2017-05-17,2017-05-18,...,Heartland Honda,Springdale,AR,72762-4877,US,OKAR,0,0,2017,5


#### Convert categorical features to numerical

In [10]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical_cols():    
    categorical_col_names = ['SalesOffice', 'Order_Type', 
       'Overall_Credit_Status', 'Delivery_Block', 'Billing_Block',
       'Block_flag', 'Line_Req_Deliv_Date', 'Confirmed_Line_Quantity',
       'Confirmed_Line_Value', 'Overall_Credit_Status_Desc',
       'Net_Value_Actual', 'ProductCategory', 'MasterSKU', 'Color',
       'CurrentFlag', 'CustomerID', 'Customer', 'City', 'State', 'Zipcode',
       'Country', 'District']
    le = LabelEncoder()
    for c in categorical_col_names:
        le.fit(data[c].values)
        data[c] = le.transform(data[c])

In [11]:
encode_categorical_cols()

In [12]:
data.sample(5)

,Sales_document,Sold_to_Party,SalesOffice,Order_Type,Unit_Price,Order_Qty,Document_Date,Req_Delivery_Date,Goods_Issue_Date,Delivery_Date,...,Customer,City,State,Zipcode,Country,District,waiting_days,wait,year,month
224426,0010335698,0000101539,1,2,203.28,2.0,2017-07-12,2017-10-17,2017-10-16,2017-10-17,...,1013,2651,58,3934,5,59,97,1,2017,7
544430,0011051423,0000101859,1,2,129.99,1.0,2018-03-26,2018-03-26,2018-03-28,2018-03-29,...,1879,1586,58,4478,5,3,0,0,2018,3
365573,0010530037,0000100950,1,2,203.28,1.0,2017-10-13,2017-10-13,2017-10-13,2017-10-18,...,1096,1569,27,567,5,27,0,0,2017,10
327027,0010463783,0000108635,1,2,14.99,12.0,2017-09-08,2017-11-13,2018-01-22,2018-01-25,...,272,858,21,2592,5,21,66,1,2017,9
418067,0010699762,0000101021,1,2,259.99,1.0,2017-12-05,2017-12-05,2017-12-08,2017-12-11,...,3047,2406,24,3573,5,59,0,0,2017,12


#### Check Order_Qty correlation with other columns

In [47]:
data.corr()['Order_Qty']

SalesOffice                  -0.189476
Order_Type                    0.003086
Unit_Price                   -0.124980
Order_Qty                     1.000000
Document_Date                 0.015713
Req_Delivery_Date             0.018912
Goods_Issue_Date              0.019242
Delivery_Date                 0.019971
Overall_Credit_Status         0.022008
Delivery_Block                0.003717
Billing_Block                -0.002621
Block_flag                   -0.000120
Line_Req_Deliv_Date           0.018561
Confirmed_Line_Quantity       0.830864
Confirmed_Line_Value          0.549205
Overall_Credit_Status_Desc    0.001540
Net_Value_Actual              0.550204
ProductCategory              -0.103002
MasterSKU                    -0.012743
Color                        -0.005755
CurrentFlag                        NaN
CustomerID                   -0.040781
Customer                     -0.054027
City                          0.014029
State                         0.000298
Zipcode                  

### Segment dealers by average reordering patterns (days between reorders) and check correlation
- Calculate each dealer's average order interval (excluding 0-day interval -- same day orders)

In [253]:
len(HYB_customer_order_intervals)

24

In [254]:
len(STD_customer_order_intervals)

4984

In [261]:
# 24 + 4984 = 5008
# found 8 dealers listed with both HYB and STD
HYB, STD = set(HYB_customer_order_intervals), set(STD_customer_order_intervals)
for customerID in HYB.intersection(STD):
    print(customerID)

0000114575
0000100974
0000106838
0000104863
0000107466
0000114269
0000101663
0000114504


In [455]:
# example of a cross-listed dealer
df.loc[df['Sold_to_Party'] == '0000101663'].sample(10)['SalesOffice']

500947    HYB
604591    HYB
572190    HYB
358068    STD
92783     STD
343718    STD
282761    STD
436720    STD
425105    STD
396628    STD
Name: SalesOffice, dtype: object

In [1]:
%store -r HYB_customer_order_intervals
%store -r STD_customer_order_intervals

In [4]:
len(HYB_customer_order_intervals)

22

In [5]:
len(STD_customer_order_intervals)

4914

In [3]:
# merge STD and HYB dealer order intervals
combined_order_intervals = {**HYB_customer_order_intervals, **STD_customer_order_intervals}
len(combined_order_intervals)

4936

In [8]:
# drop 0-day intervals in reorder interval history
# then calculate each dealer's average order interval

import sys

def calculate_avg_order_interval():
    
    avg_order_intervals = {}
    i = 0
    
    for dealer, interval in combined_order_intervals.items():    
        
        num_of_zero = len(interval) - np.count_nonzero(interval)
        m = np.ma.masked_values(interval,0)
        
        if num_of_zero != len(interval):    # ignore 0-day intervals and calculate avg
            avg_order_intervals[dealer] = np.ma.mean(m)
        else:                               # if all intervals are 0, avg = 0
            avg_order_intervals[dealer] = 0
        
        sys.stdout.write('\r'+"{0:.3%}".format(i/len(combined_order_intervals)))
        sys.stdout.flush()
        i += 1
        
    print('\n',len(avg_order_intervals))
    return avg_order_intervals

In [9]:
result = calculate_avg_order_interval()

99.980%
 4936


In [14]:
dealer_avg_order_intervals = copy.deepcopy(result)
%store dealer_avg_order_intervals

Stored 'dealer_avg_order_intervals' (dict)


In [10]:
# result = {'customerID': avg_interval_between_reorders}
dict(list(result.items())[:20])

{'0000100395': 10.788753799392097,
 '0000100710': 1.8351373855120732,
 '0000100812': 4.331838565022421,
 '0000100873': 6.542857142857143,
 '0000100974': 25.44578313253012,
 '0000101399': 5.113907284768212,
 '0000101663': 6.986552567237164,
 '0000101813': 6.145785876993166,
 '0000102173': 8.857142857142858,
 '0000102433': 11.011513157894736,
 '0000102853': 5.510518934081347,
 '0000103430': 10.858356940509916,
 '0000103648': 5.940695296523518,
 '0000104863': 11.838862559241706,
 '0000106838': 11.089820359281438,
 '0000106889': 7.9125,
 '0000107397': 9.836363636363636,
 '0000107675': 5.690140845070423,
 '0000114269': 4.627118644067797,
 '0000114504': 1.8819018404907975}

In [11]:
# generate a list of all order intervals
avg_order_intervals = np.fromiter(result.values(), dtype=float)
avg_order_intervals.size

4936

In [12]:
# generate a list of all customerIDs
reorder_customers = np.fromiter(result.keys(), dtype=int)
reorder_customers.size

4936

### Segment dealers by average reorder intervals (days)

In [341]:
# initial KMeans clustering centoids
# used kernel density function to segment 1d data instead
cluster_reorder_intervals()

array([[ 57.97509041],
       [183.43805307],
       [ 15.42508121],
       [282.36020592],
       [ 86.66615374],
       [ 35.55042117],
       [405.92822967],
       [127.46443751]])

In [539]:
# Segment reorder intervals based on KDE functions.
# Histogram and KDE comes from dwh/reorder_cadence.ipynb, 
# graphs ``kernel density estimate for HYB re-order intervals"
# and ``kernel density estimate for STD re-order intervals"

def segment_reorder_cadence():
    
    local_df = data.copy()
    local_df['reorder_interval_group'] = ''
    orders = {**HYB_orders, **STD_orders}
    
    for dealer, order_hist in orders.items():
        try:
            local_df.loc[local_df.Sold_to_Party==dealer,['reorder_interval_group']] = choose_group(dealer)
        except KeyError:
            print(dealer)
            
    return local_df


def choose_group(dealer):
    n = result[dealer]
    group = None
    if n in range(1, 14):    # bin cutoffs based on KDE functions
        group = 1
    elif n in range(14, 22):
        group = 2
    elif n in range(22, 100):
        group = 3
    else: 
        group = 4
    return group

In [540]:
temp_df = segment_reorder_cadence()

### Check correlation between Reorder_Interval_Group and Order_Qty

In [531]:
# drop any potential rows w/o valid Reorder_Interval_Group
temp_df['reorder_interval_group'].replace('', np.nan, inplace=True)
temp_df.dropna(subset=['reorder_interval_group'], inplace=True)

In [532]:
temp_df['reorder_interval_group'].isnull().any()

False

In [533]:
# convert from string (dtype='O') to float64
temp_df['reorder_interval_group'] = temp_df['reorder_interval_group'].astype(float)

In [537]:
temp_df.groupby('reorder_interval_group').corr()

Confirmd_Qty_Item  \
reorder_interval_group                                              
1.0                    Confirmd_Qty_Item                 1.000000   
                       Confirmed_Line_Quantity           1.000000   
                       Confirmed_Line_Value              0.899696   
                       Confirmed_Qty_Schl_Line           1.000000   
                       Net_Value                         0.899696   
                       Net_Value_Actual                  0.895455   
                       Net_Value_item_level              0.895455   
                       Order_Qty                         0.997802   
                       Original_Promised_Qty             0.939129   
                       Unit_Price                       -0.156106   
                       waiting_days                     -0.039075   
2.0                    Confirmd_Qty_Item                 1.000000   
                       Confirmed_Line_Quantity           1.000000   
                       Confirmed_Line_Value              0.480663   
                       Confirmed_Qty_Schl_Line           1.000000   
                       Net_Value                         0.480663   
                       Net_Value_Actual                  0.480525   
                       Net_Value_item_level              0.480525   
                       Order_Qty                         0.999829   
                       Original_Promised_Qty             0.792524   
                       Unit_Price                       -0.221071   
                       waiting_days                      0.023844   
3.0                    Confirmd_Qty_Item                 1.000000   
                       Confirmed_Line_Quantity           1.000000   
                       Confirmed_Line_Value              0.550287   
                       Confirmed_Qty_Schl_Line           0.999953   
                       Net_Value                         0.550287   
                       Net_Value_Actual                  0.549081   
                       Net_Value_item_level              0.549081   
                       Order_Qty                         0.999463   
                       Original_Promised_Qty             0.449714   
                       Unit_Price                       -0.210154   
                       waiting_days                      0.022041   
4.0                    Confirmd_Qty_Item                 1.000000   
                       Confirmed_Line_Quantity           0.999878   
                       Confirmed_Line_Value              0.772830   
                       Confirmed_Qty_Schl_Line           0.999854   
                       Net_Value                         0.772830   
                       Net_Value_Actual                  0.766143   
                       Net_Value_item_level              0.766143   
                       Order_Qty                         0.997672   
                       Original_Promised_Qty             0.391407   
                       Unit_Price                       -0.128597   
                       waiting_days                      0.016871   

                                                Confirmed_Line_Quantity  \
reorder_interval_group                                                    
1.0                    Confirmd_Qty_Item                       1.000000   
                       Confirmed_Line_Quantity                 1.000000   
                       Confirmed_Line_Value                    0.899696   
                       Confirmed_Qty_Schl_Line                 1.000000   
                       Net_Value                               0.899696   
                       Net_Value_Actual                        0.895455   
                       Net_Value_item_level                    0.895455   
                       Order_Qty                               0.997802   
                       Original_Promised_Qty                   0.939129   
                       Unit_Price               

In [535]:
# mean and variance w/in each group
temp_df.groupby('reorder_interval_group')['Order_Qty'].mean()

reorder_interval_group
1.0     8.738225
2.0    11.112306
3.0     7.500617
4.0    17.795212
Name: Order_Qty, dtype: float64

In [536]:
temp_df.groupby('reorder_interval_group')['Order_Qty'].var()

reorder_interval_group
1.0     837.731043
2.0     501.126128
3.0     395.244167
4.0    7295.575295
Name: Order_Qty, dtype: float64

In [ ]:
temp_df['reorder_interval_group'].replace('', np.nan, inplace=True)
temp_df.dropna(subset=['reorder_interval_group'], inplace=True)
temp_df.reorder_interval_group.apply(int)

### Correlation between reorder interval grouping and order quantity

In [587]:
temp_df['Order_Qty'].corr(temp_df['reorder_interval_group'])

0.013812904173032919